# AI 201 Programming Assignment 1
## A* Algorithm Implementation

Submitted by: 
Jan Lendl R. Uy, 2019-00312

### Notebook Prerequisites
Install the following Python modules, if not yet installed:
1. numpy

In [174]:
import copy

In [175]:
# CONSTANTS
INPUT_FILE_PATH = "astar_in.txt"
MAX_ALGORITHM_ITERATIONS = 50

## File Handling
This function reads an input .txt file containing the start and goal states of the puzzle then converts the puzzle contents into a nested tuple of tile values. The input file is stored in the same directory as the notebook file with the filename of "astar_in.txt".

In [176]:
def read_file(path):
    
    with open(path, "r") as file:
        lines = file.readlines()

    # Initialize lists to store the start state and goal state
    start_state = []
    goal_state = []

    # Initialize a flag to keep track if following tiles belong to the start
    # or the goal state
    reading_part = None

    for line in lines:
        line = line.strip() # Remove leading/trailing whitespace
        if line == "start":
            reading_part = "start"
        elif line == "goal":
            reading_part = "goal"
        elif line and reading_part:
            # Convert line into a list, handling '*' and converting numbers to integers
            line_list = [int(x) if x.isdigit() else x for x in line.split()]
            if reading_part == "start":
                start_state.append(line_list)
            elif reading_part == "goal":
                goal_state.append(line_list)
                
    start_state = tuple(tuple(row) for row in start_state)
    goal_state = tuple(tuple(row) for row in goal_state)

    return start_state, goal_state

start, goal = read_file(INPUT_FILE_PATH)
print(f"Start State: {start}")
print(f"Goal State: {goal}")

Start State: (('*', 1, 3), (8, 2, 4), (7, 6, 5))
Goal State: ((1, 2, 3), (8, '*', 4), (7, 6, 5))


## PuzzleBoard
This class encapsulates the variable of interest of the 8-puzzle, which include the following:

- Board contents
- Coordinates of the empty tile
- Valid movements

Among the operators overridden in this class are the \__str\__ method to "prettify" the printing of the 8-puzzle and the \__eq\__ and \__hash\__ for checking of equal board contents. Aside from this, the following methods were implemented to simplify the operations of the game:

- get_empty_tile_coords: Retrieves the coordinates of the empty space, with the origin set as the top leftmost tile
- get_valid_movements: Checks for all the possible tile movements based on the coordinates of the empty space
- add_coordinates: Returns the sum of two coordinates for tile movement tracking
- swap: Swaps the location of two entities in the puzzle, thus simulating the movement of tiles since the other entity is always the empty space

In [177]:
class PuzzleBoard:
    
    def __init__(self, board_as_2d_tuple):
        self.board_contents = board_as_2d_tuple
        self.empty_tile_coords = self.get_empty_tile_coords()
        self.valid_movements = self.get_valid_movements()   
        
        self.board_in_str = ""
        for row in self.board_contents:
            # print(f"row = {row}")
            for tile in row:
                # print(f"tile = {tile}")
                self.board_in_str += f"| {tile} "
            self.board_in_str += "|\n"
       
    # Print override
    # Formats the Puzzle printing as a 3x3 grid-like structure 
    def __str__(self):
        return self.board_in_str
    
    # Equality override
    # Checks if two boards have exactly the same elements in 
    # the same location 
    def __eq__(self, other_puzzle):
        return isinstance(other_puzzle, PuzzleBoard) and self.board_contents == other_puzzle.board_contents
    
    # Hash override
    # Uses board contents as basis for equality check
    def __hash__(self):
        return hash(tuple(self.board_contents))
    
    def get_board_as_string(self):
        return self.board_in_str 
    
    def add_coordinates(self, coords_1, coords_2):
        return tuple(map(lambda a, b: a + b, coords_1, coords_2))    
    
    def swap(self, coords_orig, coords_new):
        x, y = coords_orig
        new_x, new_y = coords_new
        board_copy = list(list(row) for row in self.board_contents)
        board_copy[x][y], board_copy[new_x][new_y] = board_copy[new_x][new_y], board_copy[x][y]
        return tuple(tuple(row) for row in board_copy)
        
    def get_valid_movements(self):
        # Define possible movements: up, down, left, right
        movements = {"up": (-1, 0), "down": (1, 0), "left": (0, -1), "right": (0, 1)}
        
        # Define a dictionary to store valid movements and the corresponding 
        # indices of swaps
        self.valid_movements = {}
        
        # Get valid movements
        for movement in movements:
            check_valid_movement = self.add_coordinates(self.empty_tile_coords, movements[movement])
            x_moved, y_moved = check_valid_movement
            # new_coords.append(check_valid_movement)
            if (x_moved >= 0 and x_moved < 3) and (y_moved >= 0 and y_moved < 3):
                self.valid_movements[movement] = check_valid_movement
            
        return self.valid_movements     
    
    def get_empty_tile_coords(self):
        empty_tile = None
        for row in self.board_contents:
            for tile in row:
                if tile == "*":
                    empty_tile = (self.board_contents.index(row), row.index(tile))
        return empty_tile

starting_puzzle = PuzzleBoard(start)
goal_puzzle = PuzzleBoard(goal)
print(f"Start State: \n{starting_puzzle}")
print(f"Goal State: \n{goal_puzzle}")

Start State: 
| * | 1 | 3 |
| 8 | 2 | 4 |
| 7 | 6 | 5 |

Goal State: 
| 1 | 2 | 3 |
| 8 | * | 4 |
| 7 | 6 | 5 |



## Node
This class represents the configuration of the puzzle as the A* algorithm searches for the optimal sequence leading to the goal state. It has the following attributes:

- Value: Contents of the 8-puzzle
- Parent node
- Cost functions f(s), g(s), and h(s)

Among the operators overridden in this class are the \__str\__ method which returns the "prettified" visualization of the puzzle and the \__eq\__ and \__hash\__ for checking of equal board contents. Aside from this, it has the method path() which retrieves the sequence of puzzle configurations from the starting/initial state to the goal state.

In [178]:
class Node:
    def __init__(self, value, parent=None, g=0, h=0):
        self.value = value
        self.parent = parent
        self.g = g
        self.h = h
        self.f = g + h
    
    def __str__(self):
        return self.value.get_board_as_string()
    
    def __hash__(self):
        return hash(self.value)
    
    def __eq__(self, other):
        return isinstance(other, Node) and self.value == other.value

    def path(self):
        node, pth = self, []
        while node:
            pth.append(node.value)
            node = node.parent
        return pth[::-1]

## Heuristic Functions
These functions are used for the computation of h(s) component of the total path cost function f(s). Among the heurist functions used for the A* algorithm are the number of tiles in the wrong position, Manhattan distance, and the Nilsson sequence score.

In [179]:
def number_of_tiles_in_wrong_position(current_board, goal_board):
    wrong_counter = 0
    
    for i in range(len(current_board)):
        for j in range(len(current_board[i])):
            if current_board[i][j] != goal_board[i][j]:
                wrong_counter += 1
    return wrong_counter

def manhattan_distance(current_board, goal_board):
    distance = 0
    # Flatten the boards into 1D lists
    if isinstance(current_board[0], list):
        current_board = [tile for row in current_board for tile in row]
    if isinstance(goal_board[0], list):
        goal_board = [tile for row in goal_board for tile in row]
        
    for i, tile in enumerate(current_board):
        if tile != goal_board[i] and tile != "*":  # Exclude the blank space represented by 0
            # Find the index of the current tile in the goal state.
            goal_index = goal_board.index(tile)
            # Convert the current index and goal index into (row, col) format.
            current_pos = divmod(i, 3)
            goal_pos = divmod(goal_index, 3)
            # Calculate the Manhattan distance and add it to the total.
            distance += abs(current_pos[0] - goal_pos[0]) + abs(current_pos[1] - goal_pos[1])
    return distance

def nilsson_sequence_score(current_board, goal_board):
    # Flatten the boards if they are 2D lists for easier index computation.
    if isinstance(current_board[0], list):
        current_board = [tile for row in current_board for tile in row]
    if isinstance(goal_board[0], list):
        goal_board = [tile for row in goal_board for tile in row]
        
    manhattan_dist = manhattan_distance(current_board, goal_board)
    
    # The correct clockwise ordering of the tiles around the periphery.
    correct_order = goal_board[1:3] + [goal_board[5], goal_board[8], goal_board[7], goal_board[6], goal_board[3]]
    current_order = current_board[1:3] + [current_board[5], current_board[8], current_board[7], current_board[6], current_board[3]]
    
    # Calculate sequence score.
    sequence_score = 0
    for i in range(len(correct_order)):
        if current_order[i] != correct_order[i] and current_order[i] != 0:
            # Check if the next tile is the correct one in the sequence.
            next_correct = correct_order[(i + 1) % len(correct_order)]
            if current_order[i] != next_correct:
                sequence_score += 2
    
    # Check the center square, add 1 if it is not the empty square.
    if current_board[4] != 0:
        sequence_score += 1
    
    # Multiply the sequence score by 3 and add the Manhattan distance.
    return 3 * sequence_score + manhattan_dist

# Example usage:
current_board = [
    [0, 1, 2],
    [7, 8, 3],  # 0 represents the empty tile.
    [6, 5, 4]
]
goal_board = [
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 0]
]

print(nilsson_sequence_score(current_board, goal_board))  # Output will be the Nilsson sequence score.


61


## A* Algorithm
The implementation of the A* algorithm below follows the pseudocode from Slide 21 of Lecture 3B. It follows all 8 steps in the pseudocode but with an additional code for ending the search for a maximum number of iterations when the algorithm does not converge to the goal state. The default heuristic function is the number of tiles in the wrong position. The heuristic parameter must be explicitly set if another heuristic function is desired for the search.

In [180]:
def a_star(start, goal, heuristic="number_of_tiles_in_wrong_position"):
    
    print("Running A* Algorithm")
    print(f"Start Node: \n{start}")
    print(f"Goal Node: \n{goal}")
    
    # Variables to be used throughout the algorithm
    open = set()
    closed = set()
    path_to_goal = None # Store sequence of nodes leading to goal node
    ctr = 0 # Counter to break non-converging search
    f = 0 # Total path cost
    g = 0
    h = compute_heuristic(start.board_contents, goal.board_contents, heuristic)
                    
    # print(f"Empty tile coordinates: {start.empty_tile_coords}")
    start_node = Node(start, h=h)
    goal_node = Node(goal)
        
    # Push starting node to open
    open.add(start_node)
    current_node = start_node
    
    while ctr < MAX_ALGORITHM_ITERATIONS:
        
        print(f"\nStep {ctr+1}")
            
        # If open is empty, return failure and end search
        # Else, continue search
        if len(open) == 0:
            print("No solution found.")
            break
        
        # Remove from OPEN the node whose f value is smallest and put it on a list called closed. 
        # Save this node as the value of current_node
        node_lowest_f = min(open, key=lambda node: node.f)
        current_node = node_lowest_f
        open.remove(node_lowest_f)
        closed.add(node_lowest_f)
        print(f"Current node: \n{current_node}")
        # print(f"node_lowest_f = {node_lowest_f}")
        # print(f"open = {open}")
        # print(f"closed = {closed}")
        
        # If current node is the goal node, return success
        # End search
        if (current_node == goal_node):
            print(f"Goal achieved!")
            path_to_goal = current_node.path()
            break
        
        # Expand current node then add these nodes to the list of
        # opened nodes
        successor_nodes = get_successors(current_node, goal_node, heuristic)
        if len(successor_nodes) == 0:
            continue
        
        # Put the successor nodes to the list of opened nodes
        # Print opened nodes for checking
        print(f"Opened Nodes:")
        for successor_node in successor_nodes:
            print(f"Successor node: \n{successor_node} \
                        \nf(s) = {successor_node.f} \
                        \ng(s) = {successor_node.g} \
                        \nh(s) = {successor_node.h}")
            open.add(successor_node)
                    
        ctr += 1

    return path_to_goal

def compute_heuristic(current_puzzle, goal_puzzle, heuristic_type):
    if heuristic_type != "number_of_tiles_in_wrong_position":
        current_puzzle_list = list(list(row) for row in current_puzzle)
        goal_puzzle_list = list(list(row) for row in goal_puzzle)
        if heuristic_type == "manhattan_distance":
            h = manhattan_distance(current_puzzle_list, goal_puzzle_list)
        elif heuristic_type == "nilsson_sequence_score":
            h = nilsson_sequence_score(current_puzzle_list, goal_puzzle_list)
    else:
        h = number_of_tiles_in_wrong_position(current_puzzle, goal_puzzle)
    return h
    
def get_successors(current, goal, heuristic="number_of_tiles_in_wrong_position"):
    
    successors = []
    current_board = current.value
    goal_board = goal.value
    
    h = compute_heuristic(current_board.board_contents, goal_board.board_contents, heuristic)
        
    # Move the tile according to the specified direction
    for movement in current_board.valid_movements:
        successor = current_board.swap(current_board.empty_tile_coords, current_board.valid_movements[movement])
        successor_puzzle = PuzzleBoard(successor)
        successor_node = Node(successor_puzzle,
                              parent = current,
                              g = current.g+1, 
                              h = h)
        successors.append(successor_node)
        
    return successors

In [181]:
path = a_star(starting_puzzle, goal_puzzle, "number_of_tiles_in_wrong_position")
if path:
    print(f"Path from start to goal:")
    for node in path:
        print(node)
else:
    print("No solution found.")

Running A* Algorithm
Start Node: 
| * | 1 | 3 |
| 8 | 2 | 4 |
| 7 | 6 | 5 |

Goal Node: 
| 1 | 2 | 3 |
| 8 | * | 4 |
| 7 | 6 | 5 |


Step 1
Current node: 
| * | 1 | 3 |
| 8 | 2 | 4 |
| 7 | 6 | 5 |

Opened Nodes:
Successor node: 
| 8 | 1 | 3 |
| * | 2 | 4 |
| 7 | 6 | 5 |
                         
f(s) = 4                         
g(s) = 1                         
h(s) = 3
Successor node: 
| 1 | * | 3 |
| 8 | 2 | 4 |
| 7 | 6 | 5 |
                         
f(s) = 4                         
g(s) = 1                         
h(s) = 3

Step 2
Current node: 
| 8 | 1 | 3 |
| * | 2 | 4 |
| 7 | 6 | 5 |

Opened Nodes:
Successor node: 
| * | 1 | 3 |
| 8 | 2 | 4 |
| 7 | 6 | 5 |
                         
f(s) = 6                         
g(s) = 2                         
h(s) = 4
Successor node: 
| 8 | 1 | 3 |
| 7 | 2 | 4 |
| * | 6 | 5 |
                         
f(s) = 6                         
g(s) = 2                         
h(s) = 4
Successor node: 
| 8 | 1 | 3 |
| 2 | * | 4 |
| 7 | 6 | 5 |
 

In [182]:
path = a_star(starting_puzzle, goal_puzzle, "manhattan_distance")
if path:
    print(f"Path from start to goal:")
    for node in path:
        print(node)
else:
    print("No solution found.")

Running A* Algorithm
Start Node: 
| * | 1 | 3 |
| 8 | 2 | 4 |
| 7 | 6 | 5 |

Goal Node: 
| 1 | 2 | 3 |
| 8 | * | 4 |
| 7 | 6 | 5 |


Step 1
Current node: 
| * | 1 | 3 |
| 8 | 2 | 4 |
| 7 | 6 | 5 |

Opened Nodes:
Successor node: 
| 8 | 1 | 3 |
| * | 2 | 4 |
| 7 | 6 | 5 |
                         
f(s) = 3                         
g(s) = 1                         
h(s) = 2
Successor node: 
| 1 | * | 3 |
| 8 | 2 | 4 |
| 7 | 6 | 5 |
                         
f(s) = 3                         
g(s) = 1                         
h(s) = 2

Step 2
Current node: 
| 8 | 1 | 3 |
| * | 2 | 4 |
| 7 | 6 | 5 |

Opened Nodes:
Successor node: 
| * | 1 | 3 |
| 8 | 2 | 4 |
| 7 | 6 | 5 |
                         
f(s) = 5                         
g(s) = 2                         
h(s) = 3
Successor node: 
| 8 | 1 | 3 |
| 7 | 2 | 4 |
| * | 6 | 5 |
                         
f(s) = 5                         
g(s) = 2                         
h(s) = 3
Successor node: 
| 8 | 1 | 3 |
| 2 | * | 4 |
| 7 | 6 | 5 |
 

In [183]:
path = a_star(starting_puzzle, goal_puzzle, "nilsson_sequence_score")
if path:
    print(f"Path from start to goal:")
    for node in path:
        print(node)
else:
    print("No solution found.")

Running A* Algorithm
Start Node: 
| * | 1 | 3 |
| 8 | 2 | 4 |
| 7 | 6 | 5 |

Goal Node: 
| 1 | 2 | 3 |
| 8 | * | 4 |
| 7 | 6 | 5 |


Step 1
Current node: 
| * | 1 | 3 |
| 8 | 2 | 4 |
| 7 | 6 | 5 |

Opened Nodes:
Successor node: 
| 8 | 1 | 3 |
| * | 2 | 4 |
| 7 | 6 | 5 |
                         
f(s) = 12                         
g(s) = 1                         
h(s) = 11
Successor node: 
| 1 | * | 3 |
| 8 | 2 | 4 |
| 7 | 6 | 5 |
                         
f(s) = 12                         
g(s) = 1                         
h(s) = 11

Step 2
Current node: 
| 8 | 1 | 3 |
| * | 2 | 4 |
| 7 | 6 | 5 |

Opened Nodes:
Successor node: 
| * | 1 | 3 |
| 8 | 2 | 4 |
| 7 | 6 | 5 |
                         
f(s) = 20                         
g(s) = 2                         
h(s) = 18
Successor node: 
| 8 | 1 | 3 |
| 7 | 2 | 4 |
| * | 6 | 5 |
                         
f(s) = 20                         
g(s) = 2                         
h(s) = 18
Successor node: 
| 8 | 1 | 3 |
| 2 | * | 4 |
| 7 | 6